# Bibtex

In [2]:
#from pymongo import MongoClient
import bibtexparser as bp
import pandas as pd
import numpy as np
import yaml as yml
import os
import requests
import json
import re
import sqlite3

dir_input = os.path.join(os.getcwd(), 'input')
dir_output = os.path.join(os.getcwd(), 'output')
dir_config = os.path.join(os.getcwd(), 'config')
configFileName = 'config_analisa_bibtex.yml'


In [3]:
# CARREGA O ARQUIVO DE CONFIGURAÇÃO
with open(os.path.join(dir_config, configFileName)) as f:
    configFile = yml.load(f, Loader=yml.loader.SafeLoader)

# ANALISA BIBTEX
listaArquivos = ( \
	configFile['FILE_ACM'], \
	configFile['FILE_IEE'], \
	configFile['FILE_SD' ]
	)

def bibtexToDict(arqv):
	with open(os.path.join(dir_input, arqv), encoding='utf8') as f:
		bib_database = bp.load(f)		
	return bib_database.entries_dict.values() # retorna lista de dicionarios
	
# UNION DATAFRAMES AND YML
listaDf = []
listaYml =[]
for arqv in listaArquivos:
	sourceArticles = bibtexToDict(arqv)
	listaDf.append(pd.DataFrame(sourceArticles))
	listaYml.append(yml.dump(list(sourceArticles)))

unionDf = pd.concat(listaDf)
unionYml = ''.join(listaYml)

In [4]:
def df_export(df, file_name, file_format):
	if file_format == 'CSV':
		df.to_csv(os.path.join(dir_output, file_name + '.csv'), sep=';', index=False, encoding='utf-8')		
		print('Exported ' + file_format + ' to ' + dir_output)
	elif file_format == 'JSON':
		df.to_json(os.path.join(dir_output, file_name + '.json'), orient = 'records')
		print('Exported ' + file_format + ' to ' + dir_output)
	elif file_format == 'YAML':
		with open(os.path.join(dir_output, file_name + '.yml'), mode='w',encoding='utf8') as f:
			f.write(unionYml)
			print('Exported ' + file_format + ' to ' + dir_output)	
	else:
		print('Formato não disponivel.')

df_export(unionDf,'ALL_ARTICLES', configFile['FORMATO'])

Exported CSV to g:\Impacta\GitHub\MBA_PyForDE\script_analisa_bibtex\output


# Article Impact

In [5]:
file_scimagojr = 'scimagojr 2020.csv'
file_jcs = 'jcs_2020.csv'
file_article_impact = 'ALL_Article_Impact.csv'

df_scimagojr = pd.read_csv(os.path.join(dir_input, file_scimagojr), delimiter=';', quotechar='"', header=0)
df_jcs = pd.read_csv(os.path.join(dir_input, file_jcs), delimiter=';', quotechar='"', header=0)

# Tratamento
dict_treat_data = {'-': np.nan, '': np.nan, None: np.nan, 'Not Available': np.nan}

# JCS
df_jcs.columns = df_jcs.columns.str.lower()
df_jcs.rename(columns ={'journal impact factor':'jcr_value'}, inplace=True)
df_jcs.jcr_value.replace(to_replace=dict_treat_data, inplace=True)
df_jcs.drop(df_jcs.columns[df_jcs.columns.str.contains('unnamed') == True], axis=1, inplace=True) # Remove colunas vazias no arquivo
df_jcs.drop('rank', axis=1, inplace=True)
df_jcs.drop_duplicates(inplace=True) # Linha inteira duplicada
df_jcs['full journal title'] = df_jcs['full journal title'].str.upper()
df_jcs['full journal title'] = df_jcs['full journal title'].str.strip()
df_jcs['jcr_value'].loc[df_jcs['jcr_value'].notnull()] = [number.replace(',','.') for number in df_jcs['jcr_value'] if type(number) != float]
df_jcs['jcr_value'] = pd.to_numeric(df_jcs['jcr_value'])

# SCIMAGO
df_scimagojr.columns = df_scimagojr.columns.str.lower()
df_scimagojr.rename(columns ={'sjr':'scimago_value'}, inplace=True)
df_scimagojr.scimago_value.replace(to_replace=dict_treat_data, inplace=True)
df_scimagojr.drop('rank', axis=1, inplace=True)
df_scimagojr['title'] = df_scimagojr['title'].str.upper()
df_scimagojr['title'] = df_scimagojr['title'].str.strip()
df_scimagojr['issn'].replace(to_replace=dict_treat_data, inplace=True)
df_scimagojr['scimago_value'].loc[df_scimagojr['scimago_value'].notnull()] = [number.replace(',','.') for number in df_scimagojr['scimago_value'] if type(number) != float]
df_scimagojr['scimago_value'] = pd.to_numeric(df_scimagojr['scimago_value'])



c:\Users\Saulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\Saulo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
df_scimagojr[df_scimagojr['scimago_value'].notnull()]['scimago_value']

0        62.937
1        40.949
2        37.461
3        34.573
4        32.011
          ...  
32599     0.100
32600     0.100
32601     0.100
32602     0.100
32603     0.100
Name: scimago_value, Length: 32604, dtype: float64

# Article Impact and Bibtex

JOIN SCIMAGO AND BIBTEX

In [7]:
df_bibtex = unionDf.copy()
df_bibtex.columns = df_bibtex.columns.str.lower()

df_bibtex.issn = df_bibtex.issn.replace(np.nan,'')
df_bibtex.issn = list(map(lambda x: x.replace('-',''), df_bibtex.issn))

In [8]:
# Trata scimago issn, criando colunas dinamicamente
# 00257656, 16993993, 16994019
# 19853718, 19858345, 21804249
df_treat_scimago_issn = pd.DataFrame([x.split(',') for x in df_scimagojr.issn if type(x) != float])

for col in df_treat_scimago_issn.columns:
	column_name = 'issn' + str(col)
	df_scimagojr.insert(len(df_scimagojr.columns), column_name, df_treat_scimago_issn[col])

df_scimagojr.drop(columns='issn', inplace=True)

list_issn_col = df_scimagojr.columns[df_scimagojr.columns.str.contains('issn')]
list_df_join = []

for col_name in list_issn_col:
	df_join_temp = df_bibtex.merge(df_scimagojr, how='inner', left_on='issn', right_on=col_name, suffixes=['_1','_2']) 
	print('Column: ' + col_name + ' shape:' + str(df_join_temp.shape))
	list_df_join.append(df_join_temp)

df_bibtex_scimagoj = pd.concat(list_df_join)\
	.drop_duplicates()\
		.dropna(subset=['issn'])

Column: issn0 shape:(39, 46)
Column: issn1 shape:(0, 46)
Column: issn2 shape:(0, 46)


JOIN JCS AND BIBTEX

In [9]:
df_bibtex['journal'] = df_bibtex['journal'].str.upper()
df_bibtex['journal'] = df_bibtex['journal'].str.strip()

df_bibtex_jcs = df_bibtex.merge(df_jcs, how='inner', left_on='journal', right_on='full journal title', suffixes=['_1','_2'])
df_bibtex_jcs.shape

(19, 29)

JOIN  BIBTEX_SCIMAGOJ AND BIBTEX_JCS

In [10]:
# JOIN
df_bibtex_impact = pd.concat([df_bibtex_scimagoj, df_bibtex_jcs])

FILTRO

In [11]:
# CARREGA O ARQUIVO DE CONFIGURAÇÃO
config_file_name = 'config_article_impact.yml'

with open(os.path.join(dir_config, config_file_name)) as f:
    config_file = yml.load(f, Loader=yml.loader.SafeLoader)

filter_list = ( config_file['TITLE'],\
	config_file['KEYWORDS'],\
	config_file['YEAR'],\
	config_file['TYPE_PUBLICATION'],\
	config_file['DOI'],\
	config_file['JCR_VALUE'],\
	config_file['SCIMAGO_VALUE'],\
	config_file['FORMATO']
	)

if config_file['JCR_VALUE'] == None:
	config_file['JCR_VALUE'] = np.nan

if config_file['SCIMAGO_VALUE'] == None:
	config_file['SCIMAGO_VALUE'] = np.nan	

print(filter_list)

('Big Data', None, None, None, None, None, 2.082, 'CSV')


In [12]:
# Filtro simultaneo
df_filtro_texto = df_bibtex_impact.loc[
	(df_bibtex_impact.title_1.str.contains("{}".format(config_file['TITLE']), case=False)) |
	(df_bibtex_impact.journal.str.contains("{}".format(config_file['TITLE']), case=False)) |
	(df_bibtex_impact.keywords.str.contains("{}".format(config_file['KEYWORDS']), case=False)) |
	(df_bibtex_impact.abstract.str.contains("{}".format(config_file['ABSTRACT']), case=False)) |
	(df_bibtex_impact.year.str.contains("{}".format(config_file['YEAR']), case=False)) |
	(df_bibtex_impact.type.str.contains("{}".format(config_file['TYPE_PUBLICATION']), case=False)) |
	(df_bibtex_impact.doi.str.contains("{}".format(config_file['DOI']), case=False))
	]

df_filtro_final = df_filtro_texto.loc[(df_filtro_texto['jcr_value'] == config_file['JCR_VALUE']) |
	(df_filtro_texto['scimago_value'] == config_file['SCIMAGO_VALUE'])]


In [13]:
df_filtro_final[['title_1','journal','jcr_value','scimago_value']].head(10)

,title_1,journal,jcr_value,scimago_value
0,Big Data Systems: A Software Engineering Persp...,ACM Comput. Surv.,NaN,2.082
1,Multimedia Big Data Analytics: A Survey,ACM Comput. Surv.,NaN,2.082
2,"A Survey on IoT Big Data: Current Status, 13 V...",ACM Comput. Surv.,NaN,2.082
3,SLA Management for Big Data Analytical Applica...,ACM Comput. Surv.,NaN,2.082
4,Computational Health Informatics in the Big Da...,ACM Comput. Surv.,NaN,2.082


EXPORT

In [14]:
df_export(df_filtro_final,'bibtex_impact',config_file['FORMATO'] )

Exported CSV to g:\Impacta\GitHub\MBA_PyForDE\script_analisa_bibtex\output


In [15]:
# # INSERE DADOS NO BANCO
# client = MongoClient("localhost", 27017)
# 
# db = client.Projeto
# db.projeto.insert_many(df_filtro_final.to_dict('records'))
# 

In [16]:
# CARREGA O ARQUIVO DE CONFIGURAÇÃO
config_file_name = 'config_api.yml'

with open(os.path.join(dir_config, config_file_name)) as f:
    config_file = yml.load(f, Loader=yml.loader.SafeLoader)

url_config = ( config_file['API_TOKEN_SCOPUS'],\
	config_file['API_TOKEN_IEEE'],\
	config_file['API_STRING'],\
	config_file['META_DATA'],\
	config_file['MAX_RECORDS'],\
	config_file['FORMATO']
	)

print(url_config)

('dd43a9e67059d268b0a6eaf9f3d73cd4', 'zcw5heuva2mrz4an52gf32y6', "/'data quality' AND 'big data'", 'data quality big data', 1000, 'json')


In [17]:
# 
# url = 'https://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=zcw5heuva2mrz4an52gf32y6&querytext='+config_file['API_STRING']
# url_scopus = 'http://api.elsevier.com/content/search/scopus?query='+config_file['API_STRING']+'&apiKey=dd43a9e67059d268b0a6eaf9f3d73cd4'
# ##var_union = []
# 
# resposta_hoteis = requests.request('GET', url)
# resposta_scopus = requests.request('GET', url_scopus)
# var_teste = resposta_hoteis.json()
# var_teste_scopus = resposta_scopus.json()
# var_teste.items()
# var_teste_scopus.items()
# 

# Consuming API IEEE

In [18]:
query_params = {
	"apikey": config_file['API_TOKEN_IEEE'],\
	"format": config_file['FORMATO'],\
	"max_records": config_file['MAX_RECORDS'],\
	"meta_data": config_file['META_DATA'],\
    "start_record": 1,\
	"sort_order": 'asc',\
	"sort_field": 'article_number'
	}
print(query_params)

#requestsIEEE = requests.get(f'http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=zcw5heuva2mrz4an52gf32y6&format=json&max_records={max_records}&start_record=1&sort_order=asc&sort_field=article_number&\
requestsIEEE = requests.get('http://ieeexploreapi.ieee.org/api/v1/search/articles?', params=query_params)

# Verificar se requestsIEEE.ok

# dict_IEEE = json.loads(requestsIEEE.text)
dict_api_ieee = requestsIEEE.json()

print('\ntotal_records: ' + str(dict_api_ieee['total_records']) +\
	'\ntotal_searched: ' + str(dict_api_ieee['total_searched']))

df_api_original_articles = pd.DataFrame.from_dict(dict_api_ieee['articles'])
df_api_articles = df_api_original_articles[['title', 'abstract', 'publication_year', 'authors', 'doi', 'content_type', 'issn', 'isbn']]


{'apikey': 'zcw5heuva2mrz4an52gf32y6', 'format': 'json', 'max_records': 1000, 'meta_data': 'data quality big data', 'start_record': 1, 'sort_order': 'asc', 'sort_field': 'article_number'}

total_records: 7339
total_searched: 5631444


In [19]:
# response headers 
pd.DataFrame.from_dict(dict(requestsIEEE.headers), orient='index')

,0
Content-Security-Policy,upgrade-insecure-requests
Content-Type,application/json;charset=utf-8
Date,"Tue, 24 May 2022 22:05:17 GMT"
Server,Mashery Proxy
Set-Cookie,JSESSIONID=5F34GlaoyToAJ7xVOGeZXDZJJxROHui4NxZ...
Strict-Transport-Security,max-age=31536000; includeSubDomains
Vary,"Origin,Access-Control-Request-Method,Access-Co..."
X-Content-Type-Options,text/html; charset=UTF-8; nosniff
X-Mashery-Responder,prod-j-worker-us-east-1d-59.mashery.com
X-XSS-Protection,1


In [20]:
# request headers 
pd.DataFrame.from_dict(dict(requestsIEEE.request.headers), orient='index')

,0
User-Agent,python-requests/2.26.0
Accept-Encoding,"gzip, deflate, br"
Accept,*/*
Connection,keep-alive


Tratamento API IEE

In [21]:
list_authors=[]

# Trata autores, dicionario com listas aninhadas
for count_articles, value in enumerate(df_api_articles['authors']):
	list_authors.append([dict['full_name'] for dict in df_api_articles['authors'][count_articles]['authors']])
	list_authors[count_articles] = '/'.join(list_authors[count_articles]) 

df_api_articles['authors'] = list_authors
df_api_articles['authors'].head(10)

C:\Users\Saulo\AppData\Local\Temp/ipykernel_19604/3012637116.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api_articles['authors'] = list_authors


0    Jae-Seung Kim/R.S. Miyaoka/R.L. Harrison/P.E. ...
1    M. Endo/T. Tsunoo/S. Kandatsu/S. Tanada/H. Ara...
2                            M. Mese/P.P. Vaidyanathan
3                    K. Karoui/R. Sammouda/M. Sammouda
4                               Xiaoyin Xu/E.L. Miller
5                                          F. Davarian
6                                 W.D. Horne/A.P. Ours
7                                M. Ella/A.R.M. Sanner
8                                            D. Carico
9          N.A. DeBardeleben/W.B. Ligon/D.C. Stanzione
Name: authors, dtype: object

In [22]:
# ISSN com '-'
issn_treated = [issn.replace('-', '') for issn in df_api_articles['issn'] if type(issn) == str]
df_api_articles['issn'].loc[df_api_articles['issn'].notna()] = issn_treated


c:\Users\Saulo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


JOIN API IEEE AND SCIMAGO

In [23]:
# Trata scimago issn, criando colunas dinamicamente
# 00257656, 16993993, 16994019
# 19853718, 19858345, 21804249
list_issn_col = df_scimagojr.columns[df_scimagojr.columns.str.contains('issn')]
list_df_join = []

for col_name in list_issn_col:
	df_join_temp = df_api_articles.merge(df_scimagojr, how='inner', left_on='issn', right_on=col_name, suffixes=[None,'_2'])\
		.dropna(subset=['issn'])	
	print('Column: ' + col_name + ' shape:' + str(df_join_temp.shape))
	list_df_join.append(df_join_temp)

df_api_articles_scimagoj = pd.concat(list_df_join).drop_duplicates()

df_api_articles_scimagoj = df_api_articles_scimagoj[['title', 'abstract', 'publication_year', 'authors', 'doi',\
	 'content_type', 'issn', 'isbn','scimago_value']]

Column: issn0 shape:(40, 29)
Column: issn1 shape:(0, 29)
Column: issn2 shape:(0, 29)


JOIN API IEEE AND JCS

In [24]:
### JOIN API IEEE AND JCS

df_api_articles['title'] = df_api_articles['title'].str.upper()
df_api_articles['title'] = df_api_articles['title'].str.strip()

df_api_articles_jcs = df_api_articles.merge(df_jcs, how='inner', left_on='title', right_on='full journal title', suffixes=[None,'_2'])
df_api_articles_jcs = df_api_articles_jcs[['title', 'abstract', 'publication_year', 'authors', 'doi',\
	 'content_type', 'issn', 'isbn','jcr_value']]

df_api_articles_jcs.shape

C:\Users\Saulo\AppData\Local\Temp/ipykernel_19604/3730687943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api_articles['title'] = df_api_articles['title'].str.upper()
C:\Users\Saulo\AppData\Local\Temp/ipykernel_19604/3730687943.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api_articles['title'] = df_api_articles['title'].str.strip()


(0, 9)

JOIN API IEEE AND SCIMAGO AND JCS

In [25]:
df_api_articles_final = pd.concat([df_api_articles_scimagoj, df_api_articles_jcs])
df_api_articles_final.head(10)

,title,abstract,publication_year,authors,doi,content_type,issn,isbn,scimago_value,jcr_value
614754,"Advances in interactive, holographic 3D displays",Reconfigurable computer generated holography -...,2002,C. Slinger/C. Cameron/S. Coomber/R. Miller/D. ...,10.1109/LEOS.2002.1133902,Conferences,10928081,0-7803-7500-9,0.124,NaN
614755,Decision on repairing and renewal of old house...,Improvement of quality of old houses is a crit...,2002,H. Sakakibara/T. Shibata,10.1109/ICSMC.2002.1176391,Conferences,1062922X,0-7803-7437-1,0.198,NaN
614756,A typology of very small companies using self-...,The world of very small companies (VSC) is cha...,2003,S. Barthelemy/J.-B. Filippi,10.1109/ICSMC.2003.1244434,Conferences,1062922X,0-7803-7952-7,0.198,NaN
614757,The JANUS experiment on ISS,In the frame of ASI supported programmes focus...,2003,R. Battaglia/V. Speziale/M. Ruggeri/A. Parabon...,10.1109/AERO.2003.1235039,Conferences,1095323X,0-7803-7651-X,0.374,NaN
614758,The Cassini solar conjunction experiment: a ne...,During the Cassini solar conjunction experimen...,2003,L. Iess/P. Tortora/J.D. Anderson/S.W. Asmar/E....,10.1109/AERO.2003.1235053,Conferences,1095323X,0-7803-7651-X,0.374,NaN
614759,Simultaneous independent voice and data (sivd)...,NaN,2003,C. Langford/R. Martin,10.1109/AERO.2003.1235096,Conferences,1095323X,0-7803-7651-X,0.374,NaN
614760,The coverage of elliptical orbits using ergodi...,One of the key performance metrics for satelli...,2004,M.W. Lo,10.1109/AERO.2004.1367638,Conferences,1095323X,0-7803-8155-6,0.374,NaN
614761,An analogue to RNS converter based on RNS prop...,The space vehicles require more and more proce...,2004,R. Lojacono,10.1109/AERO.2004.1367654,Conferences,1095323X,0-7803-8155-6,0.374,NaN
614762,IP telephony for interplanetary exploration,"Voice over IP (VoIP), using techniques develop...",2004,T. Stone/R. Alena/M. Johnson,10.1109/AERO.2004.1367720,Conferences,1095323X,0-7803-8155-6,0.374,NaN
614763,Aviation application over IPv6: performance is...,Aviation industries in United States and in Eu...,2004,V. Srivastava/C. Wargo/S. Lai,10.1109/AERO.2004.1367941,Conferences,1095323X,0-7803-8155-6,0.374,NaN


### SQLite database

In [26]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# cur.execute('''CREATE TABLE tb_articles
#                 (title text
#                  ....
#                 ,isbn text)              
#             ''')
 
dict_data_types = {
    'title': 'text',
    'abstract': 'text',
    'publication_year': 'integer',
    'authors': 'text',
    'doi': 'text',
    'content_type': 'text',
    'issn': 'text',
    'isbn': 'text',
    'scimago_value': 'real',
    'jcr_value': 'real'
    }

df_api_articles_final.to_sql('tb_articles', conn, dtype=dict_data_types, if_exists='replace', index=False)


[('Advances in interactive, holographic 3D displays',
  'Reconfigurable computer generated holography - sometimes known as electroholography - is the only technique capable of using computer held data to generate interactive, high quality, 3D images containing all the depth cues used by the human visual system. Practical applications of electroholography require computer generated hologram (CGH) patterns of between 10/sup 9/ to 10/sup 10/ pixels to be calculated and displayed at interactive rates. These pixel counts are necessary in order to generate images which are both big enough (upwards of 300 mm width) and possess a large enough field of view (FOV) to permit simultaneous, multiuser viewing. Advances in several key areas may now make such high performance electroholographic systems practical in the near future. We describe CGH design algorithms, computer architectures and CGH display engines, including the Active Tiling/spl trade/ system.',
  2002,
  'C. Slinger/C. Cameron/S. Coom

In [33]:
# cur.execute('SELECT * FROM tb_articles')
# [row for row in cur.fetchall()]

pd.read_sql_query("SELECT * FROM tb_articles LIMIT 10", conn)

,title,abstract,publication_year,authors,doi,content_type,issn,isbn,scimago_value,jcr_value
0,"Advances in interactive, holographic 3D displays",Reconfigurable computer generated holography -...,2002,C. Slinger/C. Cameron/S. Coomber/R. Miller/D. ...,10.1109/LEOS.2002.1133902,Conferences,10928081,0-7803-7500-9,0.124,None
1,Decision on repairing and renewal of old house...,Improvement of quality of old houses is a crit...,2002,H. Sakakibara/T. Shibata,10.1109/ICSMC.2002.1176391,Conferences,1062922X,0-7803-7437-1,0.198,None
2,A typology of very small companies using self-...,The world of very small companies (VSC) is cha...,2003,S. Barthelemy/J.-B. Filippi,10.1109/ICSMC.2003.1244434,Conferences,1062922X,0-7803-7952-7,0.198,None
3,The JANUS experiment on ISS,In the frame of ASI supported programmes focus...,2003,R. Battaglia/V. Speziale/M. Ruggeri/A. Parabon...,10.1109/AERO.2003.1235039,Conferences,1095323X,0-7803-7651-X,0.374,None
4,The Cassini solar conjunction experiment: a ne...,During the Cassini solar conjunction experimen...,2003,L. Iess/P. Tortora/J.D. Anderson/S.W. Asmar/E....,10.1109/AERO.2003.1235053,Conferences,1095323X,0-7803-7651-X,0.374,None
5,Simultaneous independent voice and data (sivd)...,None,2003,C. Langford/R. Martin,10.1109/AERO.2003.1235096,Conferences,1095323X,0-7803-7651-X,0.374,None
6,The coverage of elliptical orbits using ergodi...,One of the key performance metrics for satelli...,2004,M.W. Lo,10.1109/AERO.2004.1367638,Conferences,1095323X,0-7803-8155-6,0.374,None
7,An analogue to RNS converter based on RNS prop...,The space vehicles require more and more proce...,2004,R. Lojacono,10.1109/AERO.2004.1367654,Conferences,1095323X,0-7803-8155-6,0.374,None
8,IP telephony for interplanetary exploration,"Voice over IP (VoIP), using techniques develop...",2004,T. Stone/R. Alena/M. Johnson,10.1109/AERO.2004.1367720,Conferences,1095323X,0-7803-8155-6,0.374,None
9,Aviation application over IPv6: performance is...,Aviation industries in United States and in Eu...,2004,V. Srivastava/C. Wargo/S. Lai,10.1109/AERO.2004.1367941,Conferences,1095323X,0-7803-8155-6,0.374,None
